In [1]:
%run data_processing.py
# memory leaking!

23-12-27 02:03:38|MOD:data_processing|: Data Processing start!
23-12-27 02:03:38|MOD:data_processing|: 3 datas :['y', 'trade_day', 'trade_15m']


Wed Dec 27 02:03:38 2023 : y start ...
y blocks reading 10days/lag1 Data1D's ...... cost 9.05 secs
y blocks reading 20days/lag1 Data1D's ...... cost 8.39 secs
y blocks merging ...... cost 4.10 secs
y blocks process ...... cost 0.55 secs
y blocks saving  ...... cost 18.51 secs
y blocks norming ...... cost 0.00 secs
Wed Dec 27 02:04:19 2023 : y finished! Cost 40.67 Seconds
Wed Dec 27 02:04:19 2023 : trade_day start ...
trade_day blocks reading day/trade Data1D's ...... cost 46.29 secs
trade_day blocks merging ...... cost 0.12 secs
trade_day blocks process ...... cost 5.34 secs
trade_day blocks saving  ...... cost 25.05 secs
trade_day blocks norming ...... cost 34.22 secs
Wed Dec 27 02:06:10 2023 : trade_day finished! Cost 111.11 Seconds
Wed Dec 27 02:06:10 2023 : trade_15m start ...
trade_15m blocks reading 15min/trade Data1D's ...... cost 1045.64 secs
trade_15m blocks merging ...... cost 0.11 secs
trade_15m blocks process ...... cost 16.26 secs
trade_15m blocks saving  ...... cost 166.6

23-12-27 02:27:22|MOD:data_processing|: Data Processing Finished! Cost 1423.68 Seconds


... cost 42.68 secs
Wed Dec 27 02:27:21 2023 : trade_15m finished! Cost 1271.34 Seconds
